# higgsml-sklearn
Using [scikit-learn](http://scikit-learn.org/stable/) with the [Higgs Boson Machine Learning Challenge](https://higgsml.lal.in2p3.fr) dataset.

_https://github.com/adambozson/higgsml-sklearn -- Adam Bozson, June 2016_

---

## 0. Introduction
This notebook steps through the construction, validation, and scoring of a simple signal/background classifier for the HiggsML challenge.